In [1]:
from dont_patronize_me import DontPatronizeMe

# Feature
import feature.basicFeatures as bf
import feature.makeWordVector as mwv

# Preprocessing
import preprocessing.basicPreProcessing as bp

# Model
import models.deepModel as dm

# Misc for model training.
from tensorflow import keras
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import contractions

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [3]:
def ready_data(X, y):
    X = np.array(X)
    X = X.reshape(-1, 1)
    x_rus = X
    y_rus = y
    x_rus = [item[0] for item in x_rus]
    x_rus = np.array(x_rus).astype(np.float32)
    return x_rus, y_rus
    
def contract_words(text):
    """
    Removes Contractations from text. i.e. are'nt -> are not
    """
    return contractions.fix(text)


def preprocess_text(text):
    """
    Should return a list of words
    """
    text = str(text)
    text = contract_words(text)
    text = text.lower()
    text = text.replace('"', "").replace(
        ",", "").replace("'", "")
    return text.split()

# Starts Here

In [4]:
dpm = DontPatronizeMe('dataset', 'dontpatronizeme_pcl.tsv')
dpm.load_task1()
data = dpm.train_task1_df
process = bp.BasicPreProcessing()
data['text_split'] = data['text'].apply(preprocess_text)

In [5]:
def get_text_feature(text_tokens,n=[3,5,7]):
    nt = []
    for i in range(len(text_tokens)-n[0]):
        for j in n:
            nt.append(text_tokens[i:i+j])

    return nt

In [6]:
ttok = data['text_split'][0]

In [7]:
data["text_feature"] = data['text_split'].apply(get_text_feature)

In [8]:
wv = mwv.Word2VecModelTrainer().load_trained("word2vec.wordvectors")

In [9]:
basic_features = bf.BasicFeatures()
data['embeddings'] = data['text_split'].apply(
        basic_features.add_vectors, wv=wv)

In [13]:
data["embeddings_feature"] = data['text_feature'].apply(basic_features.add_vectors_multiple, wv=wv)

In [20]:
data["embeddings"].shape

(10469,)

In [23]:
# wv.most_similar(data["embeddings_feature"][1])
data["embeddings_feature"].shape

2022-01-07 18:38:50.979349: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-07 18:38:51.016573: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa329a54d80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-01-07 18:38:51.016591: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Training the model...
Epoch 1/150
237/237 [==============================] - 0s 2ms/step - loss: 0.7322 - sparse_categorical_accuracy: 0.5983
Epoch 2/150
237/237 [==============================] - 1s 3ms/step - loss: 0.6077 - sparse_categorical_accuracy: 0.6746
Epoch 3/150
237/237 [==============================] - 1s 3ms/step - loss: 0.5743 - sparse_categorical_accuracy: 0.7063
Epoch 4/150
237/237 [==============================] - 1s 3ms/step - loss: 0.5515 - sparse_categorical_accuracy: 0.7194
Epoch 5/150
237/237 [==============================] - 1s 3ms/step - loss: 0.5286 - sparse_categorical_accuracy: 0.7357
Epoch 6/150
237/237 [==============================] - 1s 3ms/step - loss: 0.5059 - sparse_categorical_accuracy: 0.7478
Epoch 7/150
237/237 [==============================] - 1s 3ms/step - loss: 0.4891 - sparse_categorical_accuracy: 0.7619
Epoch 8/150
237/237 [==============================] - 1s 4ms/step - loss: 0.4685 - sparse_categorical_accuracy: 0.7744
Epoch 9/150
237/23

KeyboardInterrupt: 

In [17]:
wv.most_similar([data["embeddings_feature"][1]])

[('glory', 0.783615231513977),
 ('cultures', 0.7813388109207153),
 ('suggesting', 0.7790683507919312),
 ('different', 0.7790452241897583),
 ('destination', 0.7770742177963257),
 ('hunger', 0.7682454586029053),
 ('minds', 0.7652076482772827),
 ('wider', 0.7584751844406128),
 ('minority', 0.7561927437782288),
 ('ways', 0.7561904191970825)]